TODO in class
* install BeautifulSoup if you do not have it already
* pip install beautifulsoup4 
* note the 4 at the end
* find two URLs that you want to scrape on SS.com
* scrape the data from the first page
* scrape the data from the second page
* you can use the functions I provided in the class

* save both as csv files
* and make plot of the data - up to you what you want to plot


* homework will be to try to srape date from some other site - such as wikipedia

In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [59]:
url = "https://www.ss.com/en/animals/dogs/"
url

'https://www.ss.com/en/animals/dogs/'

In [60]:
req = requests.get("https://www.ss.com/en/animals/dogs/")
req.status_code

200

In [61]:
soup = BeautifulSoup(req.text, 'lxml') 
soup.title

<title>SS.COM Animals - Dogs, puppies, Prices - Advertisements</title>

In [62]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [63]:
column_names = getColList(soup)
column_names

['description', 'url', 'Breed', 'Age', 'Price']

In [64]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [65]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [66]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist] # so return a list of dictionaries

In [67]:
# so with this function I can get full dataframe from a single page on ss.com not only apartments
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(url)
    rowlist = getRowList(url)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [69]:
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.shape

(30, 5)

In [70]:
dtemp.head()

,description,url,Breed,Age,Price
0,"Mamma Vācu aitu sugas, tēvs Haski. \r\nLabs sa...",https://ss.com/msg/en/animals/dogs/outbred/apx...,Outbred,1.1 mon.,50 €
1,"Весёлый, ласковый мальчик ждёт любящую семью. ...",https://ss.com/msg/en/animals/dogs/caniche-min...,Caniche Miniature,3 mon.,700 €
2,Pēc mūsu 13 gadus vecā Labradora zaudēšana pag...,https://ss.com/msg/en/animals/dogs/labrador-re...,Labrador Retriever,3 mon.,buy
3,"Nodosim mīļās, gādīgās, labās rokās mazos suņu...",https://ss.com/msg/en/animals/dogs/labrador-re...,Labrador Retriever,2 mon.,100 €
4,"Очаровательные, любознательные малыши ищут отв...",https://ss.com/msg/en/animals/dogs/outbred/cbe...,Outbred,3 mon.,-


In [56]:
dtemp.to_excel("dogs_data.xlsx")

In [68]:
dogs = "https://www.ss.com/en/animals/dogs/"
idf = getDFfromURL(dogs)
idf.head()

TypeError: slice indices must be integers or None or have an __index__ method